In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from lightgbm import LGBMClassifier
from optuna.integration import OptunaSearchCV
from optuna.distributions import IntDistribution, FloatDistribution, CategoricalDistribution
from datetime import datetime
from joblib import dump
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
path_data = '/home/arfrech/Documents/Cursos/data science/Projects/cybersecurity - TCC/tcc-dsa/data/processed'
path_interim = '/home/arfrech/Documents/Cursos/data science/Projects/cybersecurity - TCC/tcc-dsa/data/interim'
path_model = '/home/arfrech/Documents/Cursos/data science/Projects/cybersecurity - TCC/tcc-dsa/models'
df = pd.read_parquet(f'{path_data}/Obfuscated-MalMem2022-processed.parquet')
df.head(3)

,pslist.nproc,pslist.nppid,pslist.avg_threads,pslist.avg_handlers,dlllist.ndlls,dlllist.avg_dlls_per_proc,handles.nhandles,handles.avg_handles_per_proc,handles.nfile,handles.nevent,...,svcscan.fs_drivers,svcscan.process_services,svcscan.shared_process_services,svcscan.nactive,callbacks.ncallbacks,callbacks.nanonymous,callbacks.ngeneric,Class,Category,Family
0,45,17,10.555556,202.844444,1694,38.50000,9129,212.302326,670,3161,...,26,24,116,121,87,0,8,Benign,Benign,Benign
1,47,19,11.531915,242.234043,2074,44.12766,11385,242.234043,840,3761,...,26,24,118,122,87,0,8,Benign,Benign,Benign
2,40,14,14.725000,288.225000,1932,48.30000,11529,288.225000,1050,3996,...,26,27,118,120,88,0,8,Benign,Benign,Benign


In [3]:
# targets = ['Class','Category','Family']
# models = ['knn','svm','lgbm']
# results_df = pd.DataFrame(columns=['target','model','accuracy','precision','recall','f1','train_time_s'])
# params_dict, predicoes = dict(), dict()

# for tgt in targets:

#     if tgt=='Class':
#         X = df.iloc[:,:-3].values
#         y = df.loc[:,tgt].values.codes
#     else:
#         X = df.query('Class!="Benign"').iloc[:,:-3].values
#         y = df.query('Class!="Benign"').loc[:,tgt].values.codes
#     train_x,test_x,train_y,test_y = train_test_split(X,y,test_size=0.2,random_state=9587)
#     cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=5,random_state=9587)
#     predicoes[f'cod_{tgt.lower()}'] = dict(enumerate(df.loc[:,tgt].cat.categories))
#     predicoes[f'y_{tgt.lower()}'] = y

#     for model_name in models:

#         def objective(trial):
#             if model_name=='knn':
#                 params = {'weights':'uniform',
#                         'metric':'euclidean',
#                         'n_neighbors':trial.suggest_int('n_neighbors',1,101)}
#                         #   'weights':trial.suggest_categorical('weights',['uniform','distance']),
#                         #   'metric':trial.suggest_categorical('metric',['euclidean','manhattan','minkowski'])}                 
#                 pipeline = Pipeline([('scaler',StandardScaler()),('classifier',KNeighborsClassifier(**params))])
            
#             elif model_name=='svm':           
#                 params = {'kernel':'linear',
#                         'C':trial.suggest_float('C',1e-5,1e5,log=True)}
#                         #   'kernel':trial.suggest_categorical('kernel',['linear','poly','rbf','sigmoid']),
#                         #   'gamma':trial.suggest_float('gamma',1e-5,1e5,log=True),
#                         #   'degree':trial.suggest_int('degree',1,5),
#                         #   'coef0':trial.suggest_float('coef0',-1.0,1.0)}
#                 pipeline = Pipeline([('scaler',StandardScaler()),('classifier',SVC(**params))])
            
#             elif model_name=='lgbm':
#                 params = {'verbosity':-1,
#                           'boosting_type':'gbdt',
#                           'num_leaves':trial.suggest_int('num_leaves',2,256),
#                           'min_child_samples':trial.suggest_int('min_child_samples',5,100),
#                           'learning_rate':trial.suggest_float('learning_rate',0.001,0.1,log=True)}
#                 pipeline = Pipeline([('scaler',StandardScaler()),('classifier',LGBMClassifier(**params))])
            
#             pipeline.fit(train_x,train_y)
#             score = cross_val_score(pipeline,train_x,train_y,scoring='accuracy',cv=cv).mean()
#             return score

#         t_initial = datetime.now()
#         n_trials = 3
#         study = optuna.create_study(direction='maximize',study_name=f'{tgt.lower()}_{model_name}_{datetime.now().strftime("%Y%m%d")}')
#         study.optimize(objective,n_trials=n_trials,show_progress_bar=True)
#         t_final = datetime.now()
#         train_time = (t_final - t_initial).total_seconds()
#         best_params = study.best_params
#         if model_name=='knn':
#             classifier = KNeighborsClassifier(**best_params)
#         elif model_name=='svm':
#             classifier = SVC(**best_params)
#         elif model_name=='lgbm':
#             classifier = LGBMClassifier(**best_params)
#         best_model = Pipeline([('scaler',StandardScaler()),('classifier',classifier)])
#         best_model.fit(train_x,train_y)
#         path_model = '/home/arfrech/Documents/Cursos/data science/Projects/cybersecurity - TCC/tcc-dsa/models'
#         dump(best_model,f'{path_model}/{tgt.lower()}_{model_name}_{datetime.now().strftime("%Y%m%d")}.joblib') 
#         yhat = best_model.predict(test_x)
#         accuracy = accuracy_score(test_y,yhat).round(5)
#         precision = precision_score(test_y,yhat,average='micro').round(5)
#         recall = recall_score(test_y,yhat,average='micro').round(5)
#         f1 = f1_score(test_y,yhat,average='micro').round(5)
#         res = pd.DataFrame(columns=results_df.columns)
#         res.loc[0,results_df.columns] = [tgt.lower(),model_name,accuracy,precision,recall,f1,train_time]
#         results_df = pd.concat([results_df,res],axis=0,ignore_index=True)
#         results_df.to_csv(f'{path_interim}/metricas_avaliacao.csv',index=False,sep=';')
#         params_dict[f'{tgt.lower()}_{model_name}'] = best_params
#         dump(params_dict,f'{path_model}/params_dict_{datetime.now().strftime("%Y%m%d")}.joblib')
#         predicoes[f'yhat_{tgt.lower()}_{model_name}'] = yhat
#         dump(predicoes,f'{path_model}/predicoes_{datetime.now().strftime("%Y%m%d")}.joblib')

In [4]:
# targets_models = [('Class','knn'),('Class','svm'),('Class','lgbm'),
#                   ('Category','knn'),('Category','svm'),('Category','lgbm'),
#                   ('Family','knn'),('Family','svm'),('Family','lgbm')]
# results_df = pd.DataFrame(columns=['target','model','accuracy','precision','recall','f1','train_time_s'])
# params_dict, predicoes = dict(), dict()

# for tgt,model_name in tqdm(targets_models):

#     if tgt=='Class':
#         X = df.iloc[:,:-3].values
#         y = df.loc[:,tgt].values.codes
#     else:
#         X = df.query('Class!="Benign"').iloc[:,:-3].values
#         y = df.query('Class!="Benign"').loc[:,tgt].values.codes
#     train_x,test_x,train_y,test_y = train_test_split(X,y,test_size=0.2,random_state=9587)
#     cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=5,random_state=9587)
#     predicoes[f'cod_{tgt.lower()}'] = dict(enumerate(df.loc[:,tgt].cat.categories))
#     predicoes[f'y_{tgt.lower()}'] = test_y

#     if model_name=='knn':
#         classifier = KNeighborsClassifier()
#         param_grid = {'classifier__n_neighbors':[1,11,101]} #range(1,100,2)}
    
#     elif model_name=='svm':
#         classifier = LinearSVC()
#         param_grid = {'classifier__dual':['auto'],
#                       'classifier__random_state':[9587],
#                       'classifier__max_iter':[10000],
#                       'classifier__C':[10]} 

#     elif model_name=='lgbm': 
#         classifier = LGBMClassifier()
#         param_grid = {'classifier__verbose': [-1],
#                       'classifier__learning_rate': [0.001,0.01],
#                       'classifier__num_leaves': [31,127],
#                       'classifier__min_child_samples': [1,10]} 
        
#     pipeline = Pipeline([('scaler',StandardScaler()),('classifier',classifier)])
#     grid_search = GridSearchCV(pipeline,param_grid,cv=cv,scoring='accuracy')
    
#     t_initial = datetime.now()
#     grid_search.fit(train_x,train_y)
#     t_final = datetime.now()

#     dump(grid_search,f'{path_model}/{tgt.lower()}_{model_name}_{datetime.now().strftime("%Y%m%d")}.joblib') 

#     best_params = grid_search.best_params_
#     params_dict[f'{tgt.lower()}_{model_name}'] = best_params
#     dump(params_dict,f'{path_model}/params_dict_{datetime.now().strftime("%Y%m%d")}.joblib')

#     yhat = grid_search.predict(test_x)
#     predicoes[f'yhat_{tgt.lower()}_{model_name}'] = yhat
#     dump(predicoes,f'{path_model}/predicoes_{datetime.now().strftime("%Y%m%d")}.joblib')

#     accuracy = accuracy_score(test_y,yhat)
#     precision = precision_score(test_y,yhat,average='micro')
#     recall = recall_score(test_y,yhat,average='micro')
#     f1 = f1_score(test_y,yhat,average='micro')
#     train_time = (t_final - t_initial).total_seconds()
#     res = pd.DataFrame(columns=results_df.columns)
#     res.loc[0,results_df.columns] = [tgt.lower(),model_name,accuracy,precision,recall,f1,train_time]
#     results_df = pd.concat([results_df,res],axis=0,ignore_index=True)
#     results_df.to_csv(f'{path_interim}/metricas_avaliacao.csv',index=False,sep=';')

In [6]:
# targets_models = [('Class','knn'),('Category','knn'),('Family','knn'),
#                   ('Class','svm'),('Category','svm'),('Family','svm'),
#                   ('Class','lgbm'),('Category','lgbm'),('Family','lgbm')]
# results_df = pd.DataFrame(columns=['target','model','accuracy','precision','recall','f1','train_time_s'])
# params_dict, predicoes, falhas = dict(), dict(), list()
# n_trials = 100

# for tgt,model_name in tqdm(targets_models):
#     try:
#         if tgt=='Class':
#             X = df.iloc[:,:-3].values
#             y = df.loc[:,tgt].values.codes
#         else:
#             X = df.query('Class!="Benign"').iloc[:,:-3].values
#             y = df.query('Class!="Benign"').loc[:,tgt].values.codes
#         num_class = len(np.unique(y))
#         train_x,test_x,train_y,test_y = train_test_split(X,y,test_size=0.2,random_state=9587)
#         cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=5,random_state=9587)
#         predicoes[f'cod_{tgt.lower()}'] = dict(enumerate(df.loc[:,tgt].cat.categories))
#         predicoes[f'y_{tgt.lower()}'] = test_y
        
#         if model_name=='knn':
#             classifier = KNeighborsClassifier()
#             param_grid = {'n_neighbors':IntDistribution(1,1000,step=1),
#                         'weights':CategoricalDistribution(['uniform','distance']),
#                         'algorithm':CategoricalDistribution(['ball_tree','kd_tree','brute']),
#                         'leaf_size':IntDistribution(10,100,step=10),
#                         'p':CategoricalDistribution([1,2,np.inf])}
        
#         elif model_name=='svm':
#             classifier = SVC()
#             param_grid = {'C':FloatDistribution(1e-5,1e5,log=True),
#                         'kernel':CategoricalDistribution(['linear','poly','rbf','sigmoid']),
#                         'degree':IntDistribution(2,10,step=1),
#                         'gamma':FloatDistribution(1e-5,1e2,log=True),
#                         'coef0':IntDistribution(-1, 1),
#                         'shrinking':CategoricalDistribution([True, False]),
#                         'tol': FloatDistribution(1e-5,1e-1,log=True),
#                         'class_weight':CategoricalDistribution([None, 'balanced']),
#                         'max_iter': IntDistribution(100,1000,step=100),
#                         'decision_function_shape':CategoricalDistribution(['ovo', 'ovr']),
#                         'random_state':CategoricalDistribution([9587])}

#         elif model_name=='lgbm': 
#             classifier = LGBMClassifier()
#             param_grid = {'boosting':CategoricalDistribution(['gbdt','rf','dart']),
#                         'objective':CategoricalDistribution(['multiclass','ovr']),
#                         'num_class':CategoricalDistribution([num_class]),
#                         'random_state':CategoricalDistribution([9587]),
#                         'verbose':CategoricalDistribution([-1]),
#                         'learning_rate':FloatDistribution(1e-5,1e0,log=True),
#                         'n_estimators':IntDistribution(50,1000,step=10),
#                         'max_depth':IntDistribution(3,15),
#                         'num_leaves':IntDistribution(2,50),
#                         'min_child_samples':IntDistribution(1,20),
#                         'subsample':FloatDistribution(0.1,1.0),
#                         'colsample_bytree':FloatDistribution(0.1,1.0),
#                         'reg_alpha':FloatDistribution(1e-9,10.0,log=True),
#                         'reg_lambda':FloatDistribution(1e-9,10.0,log=True),}
            
#         optuna_search = OptunaSearchCV(estimator=classifier,
#                                     param_distributions=param_grid,
#                                     cv=cv,
#                                     scoring='accuracy',
#                                     error_score='raise',
#                                     n_trials=n_trials,
#                                     random_state=9587,
#                                     refit=True,
#                                     verbose=-1,
#                                     n_jobs=3)
        
#         pipeline = Pipeline([('scaler',StandardScaler()),('classifier',optuna_search)])
        
#         t_initial = datetime.now()
#         pipeline.fit(train_x,train_y)
#         t_final = datetime.now()

#         dump(pipeline,f'{path_model}/{tgt.lower()}_{model_name}_{datetime.now().strftime("%Y%m%d")}.joblib') 

#         best_params = pipeline['classifier'].best_params_
#         params_dict[f'{tgt.lower()}_{model_name}'] = best_params
#         dump(params_dict,f'{path_model}/params_dict_{datetime.now().strftime("%Y%m%d")}.joblib')

#         yhat = pipeline.predict(test_x)
#         predicoes[f'yhat_{tgt.lower()}_{model_name}'] = yhat
#         dump(predicoes,f'{path_model}/predicoes_{datetime.now().strftime("%Y%m%d")}.joblib')

#         accuracy = accuracy_score(test_y,yhat)
#         precision = precision_score(test_y,yhat,average='micro')
#         recall = recall_score(test_y,yhat,average='micro')
#         f1 = f1_score(test_y,yhat,average='micro')
#         train_time = (t_final - t_initial).total_seconds()
#         res = pd.DataFrame(columns=results_df.columns)
#         res.loc[0,results_df.columns] = [tgt.lower(),model_name,accuracy,precision,recall,f1,train_time]
#         results_df = pd.concat([results_df,res],axis=0,ignore_index=True)
#         results_df.to_csv(f'{path_interim}/metricas_avaliacao.csv',index=False,sep=';')
#     except:
#         falhas.append(tgt+'_'+model_name)

  0%|          | 0/9 [00:00<?, ?it/s]

[I 2023-12-28 09:54:44,938] A new study created in memory with name: no-name-aff3cfbb-1c59-4ef5-80a7-a39553cd9029
[I 2023-12-28 10:04:58,757] Trial 0 finished with value: 0.9960462621159394 and parameters: {'n_neighbors': 544, 'weights': 'distance', 'algorithm': 'brute', 'leaf_size': 10, 'p': 1}. Best is trial 0 with value: 0.9960462621159394.
[I 2023-12-28 10:10:36,377] Trial 1 finished with value: 0.993100320692445 and parameters: {'n_neighbors': 190, 'weights': 'uniform', 'algorithm': 'kd_tree', 'leaf_size': 20, 'p': 2}. Best is trial 0 with value: 0.9960462621159394.
[I 2023-12-28 10:20:56,520] Trial 2 finished with value: 0.9899907207519566 and parameters: {'n_neighbors': 658, 'weights': 'uniform', 'algorithm': 'ball_tree', 'leaf_size': 70, 'p': 1}. Best is trial 0 with value: 0.9960462621159394.
[I 2023-12-28 10:27:23,638] Trial 3 finished with value: 0.9949264635360984 and parameters: {'n_neighbors': 601, 'weights': 'distance', 'algorithm': 'ball_tree', 'leaf_size': 20, 'p': 2}.

In [8]:
falhas

['Category_svm', 'Family_svm', 'Class_lgbm']

In [9]:
results_df

,target,model,accuracy,precision,recall,f1,train_time_s
0,class,knn,0.999828,0.999828,0.999828,0.999828,43388.088162
1,category,knn,0.667071,0.667071,0.667071,0.667071,9330.313615
2,family,knn,0.408917,0.408917,0.408917,0.408917,9072.167587
3,class,svm,0.999914,0.999914,0.999914,0.999914,5315.889962
4,category,lgbm,0.772901,0.772901,0.772901,0.772901,101965.594856
5,family,lgbm,0.540076,0.540076,0.540076,0.540076,250316.604496


In [5]:
from model_tuning import *

In [ ]:
targets_models = [('Class','knn'),('Category','knn'),('Family','knn'),
                  ('Class','svm'),('Category','svm'),('Family','svm'),
                  ('Class','lgbm'),('Category','lgbm'),('Family','lgbm')]
results_df = pd.DataFrame()
n_trials = 3

for tgt,model_name in tqdm(targets_models):
    if tgt=='Class':
        X = df.iloc[:,:-3].values
        y = df.loc[:,tgt].values.codes
    else:
        X = df.query('Class!="Benign"').iloc[:,:-3].values
        y = df.query('Class!="Benign"').loc[:,tgt].values.codes
    classifier_tuner = ClassifierTuner(X, y)
    classifier_tuner.optimize_models(n_trials=n_trials)
    results = classifier_tuner.get_results()
    results_df = pd.concat([results_df,results],axis=0,ignore_index=True)

In [6]:
y

NameError: name 'y' is not defined